<a href="https://colab.research.google.com/github/MujeebShinwari/Classification-Using-ViT-EfficientNet-and-ResNet-Models-on-Fundus_CIMT_Dataset/blob/main/Classification_Using_ViT%2C_EfficientNet%2C_and_ResNet_Models_on_Fundus_CIMT_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>